In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Kirti
[nltk_data]     Mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+")   ## only word
word=set(stopwords.words("english"))

In [5]:
data_new=pd.read_csv('english_dataset.tsv', sep = '\t', encoding="latin-1")

In [6]:
data_new.shape

(5852, 5)

In [7]:
data_new=data_new.dropna()

In [8]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5852 entries, 0 to 5851
Data columns (total 5 columns):
text_id    5852 non-null object
text       5852 non-null object
task_1     5852 non-null object
task_2     5852 non-null object
task_3     5852 non-null object
dtypes: object(5)
memory usage: 274.3+ KB


In [96]:
data_new = data_new[data_new["task_3"]!= "NONE"]

In [97]:
Column_Sequence=np.array(data_new["text"],dtype="str")

In [98]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [99]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [100]:
#load pre-trained embedding:

import io
import os, re, csv, math, codecs
from tqdm import tqdm
##
#####load embeddings
print('loading word embeddings..300 dimention.')
embeddings_index = {}
f = codecs.open('wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

#GLOVE_DIR = "C\:\Users\Abhi\Desktop\Abhinav 2nd imp\Implementation_text and image\2_class"
#embeddings_index = {}
#f = open('glove.6B.300d.txt', encoding="utf8")
#for line in f:
#    values = line.split()
#    word = values[0]
#    coefs = np.asarray(values[1:], dtype='float32')
#    embeddings_index[word] = coefs
#f.close()
#print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

loading word embeddings..300 dimention.


111052it [00:19, 5796.41it/s]


found 111052 word vectors


In [101]:
##### Printing words and index pair 
index=tokenizer.word_index
#print index


###Vocabulary size
vocab_size = len(index)
#print vocab_size

In [102]:

##################################################################################
##################################################################################

#####embedding matrix
print('preparing embedding matrix...')
MAX_NB_WORDS = 800000
embed_dim = 300
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

#embedding_matrix = np.random.random((len(tokenizer.word_index) + 1, 300))
#for word, i in tokenizer.word_index.items():
#    embedding_vector = embeddings_index.get(word)
#    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
#        embedding_matrix[i] = embedding_vector
        
#embedding_layer = Embedding(len(tokenizer.word_index) + 1,300,weights=[embedding_matrix],input_length=300,trainable=True)


preparing embedding matrix...
number of null word embeddings: 1842


In [103]:
max(tokenizer.word_counts.values())

640

In [104]:
len(tokenizer.word_counts)

5301

In [105]:
X_train=X_t

In [106]:
size_embedding = max(list(tokenizer.word_index.values()))

In [107]:
X_train.shape


(2261, 300)

In [108]:
YY=data_new["task_3"]

In [109]:
YY

1       TIN
7       TIN
11      TIN
15      TIN
23      TIN
27      TIN
32      TIN
34      TIN
43      TIN
44      TIN
46      TIN
47      TIN
48      TIN
49      TIN
51      TIN
61      TIN
63      TIN
91      TIN
95      TIN
102     TIN
129     TIN
134     UNT
144     TIN
145     TIN
157     UNT
212     TIN
213     TIN
218     TIN
233     TIN
249     UNT
       ... 
5798    TIN
5799    TIN
5803    TIN
5804    UNT
5805    TIN
5821    TIN
5822    TIN
5826    TIN
5829    TIN
5830    TIN
5831    UNT
5832    UNT
5833    TIN
5834    UNT
5835    TIN
5836    UNT
5837    TIN
5838    TIN
5839    TIN
5840    TIN
5841    TIN
5842    TIN
5843    TIN
5844    TIN
5845    TIN
5846    UNT
5847    UNT
5849    TIN
5850    UNT
5851    UNT
Name: task_3, Length: 2261, dtype: object

In [110]:
#train['sentiment'] = train['sentiment'].map({'OAG': 2, 'CAG': 1,'NAG': 0})
YY = YY.map({'UNT':0, 'TIN': 1})

In [111]:
YY

1       1
7       1
11      1
15      1
23      1
27      1
32      1
34      1
43      1
44      1
46      1
47      1
48      1
49      1
51      1
61      1
63      1
91      1
95      1
102     1
129     1
134     0
144     1
145     1
157     0
212     1
213     1
218     1
233     1
249     0
       ..
5798    1
5799    1
5803    1
5804    0
5805    1
5821    1
5822    1
5826    1
5829    1
5830    1
5831    0
5832    0
5833    1
5834    0
5835    1
5836    0
5837    1
5838    1
5839    1
5840    1
5841    1
5842    1
5843    1
5844    1
5845    1
5846    0
5847    0
5849    1
5850    0
5851    0
Name: task_3, Length: 2261, dtype: int64

In [112]:
#Y_L=data_new["Comment label"]

# One-hot encode data
Y_train=np_utils.to_categorical(YY)
#uniqe_class=np.unique(All_class)

In [114]:
Y_train.shape

(2261, 2)

In [115]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.20, random_state=42)

In [116]:
y_train.shape

(1808, 2)

In [117]:
x_train.shape

(1808, 300)

In [118]:
max_len=300

In [119]:
x_train

array([[  97,  417, 2654, ...,    0,    0,    0],
       [4763, 4764, 4765, ...,    0,    0,    0],
       [ 629, 2362, 2363, ...,    0,    0,    0],
       ...,
       [ 718,  193,   55, ...,    0,    0,    0],
       [   3,    5,  502, ...,    0,    0,    0],
       [   9, 1442,  332, ...,    0,    0,    0]])

In [120]:
model = Sequential()
model.add(Embedding(size_embedding+1,100,input_length=max_len))

model.add( Convolution1D(8, 4 ))
model.add( Activation('relu') )

model.add( Convolution1D(16,4) )

model.add( Activation('relu'))
model.add(Convolution1D(64,4))
model.add( MaxPooling1D( pool_size=(4) ) )
model.add(Activation("relu"))
model.add(Convolution1D(64,4))
model.add(Activation("relu"))

model.add( Flatten() )

model.add( Dense(2) )

model.add( Activation('sigmoid') )

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 100)          530200    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 297, 8)            3208      
_________________________________________________________________
activation_6 (Activation)    (None, 297, 8)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 294, 16)           528       
_________________________________________________________________
activation_7 (Activation)    (None, 294, 16)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 291, 64)           4160      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 72, 64)            0         
__________

In [121]:
from keras.callbacks import ModelCheckpoint
filepath = "task3_cnn_fasttext.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [122]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [124]:
hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 1808 samples, validate on 453 samples
Epoch 1/100
1808/1808 [==============================] - ETA: 4s - loss: 0.4533 - acc: 0.750 - ETA: 4s - loss: 0.3560 - acc: 0.812 - ETA: 4s - loss: 0.2793 - acc: 0.854 - ETA: 4s - loss: 0.2461 - acc: 0.875 - ETA: 4s - loss: 0.2265 - acc: 0.887 - ETA: 4s - loss: 0.2104 - acc: 0.895 - ETA: 4s - loss: 0.2009 - acc: 0.897 - ETA: 4s - loss: 0.1895 - acc: 0.898 - ETA: 4s - loss: 0.1858 - acc: 0.895 - ETA: 4s - loss: 0.1809 - acc: 0.896 - ETA: 4s - loss: 0.1906 - acc: 0.897 - ETA: 4s - loss: 0.1898 - acc: 0.903 - ETA: 3s - loss: 0.2052 - acc: 0.901 - ETA: 3s - loss: 0.1959 - acc: 0.906 - ETA: 3s - loss: 0.1899 - acc: 0.910 - ETA: 3s - loss: 0.1838 - acc: 0.914 - ETA: 3s - loss: 0.1764 - acc: 0.919 - ETA: 3s - loss: 0.1751 - acc: 0.920 - ETA: 3s - loss: 0.1836 - acc: 0.917 - ETA: 3s - loss: 0.1825 - acc: 0.918 - ETA: 3s - loss: 0.1773 - acc: 0.921 - ETA: 3s - loss: 0.1772 - acc: 0.920 - ETA: 3s - loss: 0.1800 - acc: 0.919 - ETA: 2s - loss: 0.1825

1808/1808 [==============================] - ETA: 4s - loss: 0.1787 - acc: 0.968 - ETA: 4s - loss: 0.0918 - acc: 0.984 - ETA: 4s - loss: 0.0627 - acc: 0.989 - ETA: 4s - loss: 0.0476 - acc: 0.992 - ETA: 4s - loss: 0.0385 - acc: 0.993 - ETA: 4s - loss: 0.0324 - acc: 0.994 - ETA: 4s - loss: 0.0283 - acc: 0.995 - ETA: 4s - loss: 0.0285 - acc: 0.992 - ETA: 4s - loss: 0.0255 - acc: 0.993 - ETA: 4s - loss: 0.0232 - acc: 0.993 - ETA: 4s - loss: 0.0256 - acc: 0.991 - ETA: 4s - loss: 0.0242 - acc: 0.992 - ETA: 4s - loss: 0.0275 - acc: 0.990 - ETA: 3s - loss: 0.0259 - acc: 0.991 - ETA: 3s - loss: 0.0258 - acc: 0.991 - ETA: 3s - loss: 0.0249 - acc: 0.992 - ETA: 3s - loss: 0.0236 - acc: 0.992 - ETA: 3s - loss: 0.0224 - acc: 0.993 - ETA: 3s - loss: 0.0220 - acc: 0.993 - ETA: 3s - loss: 0.0210 - acc: 0.993 - ETA: 3s - loss: 0.0215 - acc: 0.992 - ETA: 3s - loss: 0.0205 - acc: 0.992 - ETA: 3s - loss: 0.0198 - acc: 0.993 - ETA: 3s - loss: 0.0192 - acc: 0.993 - ETA: 2s - loss: 0.0186 - acc: 0.993 - ETA: 

1808/1808 [==============================] - ETA: 4s - loss: 7.2124e-04 - acc: 1.000 - ETA: 4s - loss: 0.0720 - acc: 0.9844    - ETA: 4s - loss: 0.0483 - acc: 0.989 - ETA: 4s - loss: 0.0364 - acc: 0.992 - ETA: 4s - loss: 0.0294 - acc: 0.993 - ETA: 4s - loss: 0.0249 - acc: 0.994 - ETA: 4s - loss: 0.0223 - acc: 0.995 - ETA: 4s - loss: 0.0199 - acc: 0.996 - ETA: 4s - loss: 0.0182 - acc: 0.996 - ETA: 4s - loss: 0.0168 - acc: 0.996 - ETA: 4s - loss: 0.0156 - acc: 0.997 - ETA: 4s - loss: 0.0146 - acc: 0.997 - ETA: 4s - loss: 0.0136 - acc: 0.997 - ETA: 4s - loss: 0.0218 - acc: 0.995 - ETA: 3s - loss: 0.0248 - acc: 0.993 - ETA: 3s - loss: 0.0234 - acc: 0.994 - ETA: 3s - loss: 0.0221 - acc: 0.994 - ETA: 3s - loss: 0.0209 - acc: 0.994 - ETA: 3s - loss: 0.0199 - acc: 0.995 - ETA: 3s - loss: 0.0189 - acc: 0.995 - ETA: 3s - loss: 0.0181 - acc: 0.995 - ETA: 3s - loss: 0.0175 - acc: 0.995 - ETA: 3s - loss: 0.0233 - acc: 0.993 - ETA: 3s - loss: 0.0224 - acc: 0.993 - ETA: 3s - loss: 0.0216 - acc: 0.993

1808/1808 [==============================] - ETA: 4s - loss: 0.0016 - acc: 1.000 - ETA: 4s - loss: 0.0012 - acc: 1.000 - ETA: 4s - loss: 0.0012 - acc: 1.000 - ETA: 4s - loss: 0.0013 - acc: 1.000 - ETA: 4s - loss: 0.0012 - acc: 1.000 - ETA: 4s - loss: 0.0012 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0010 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0010 - acc: 1.000 - ETA: 3s - loss: 0.0010 - acc: 1.000 - ETA: 3s - loss: 9.8085e-04 - acc: 1.000 - ETA: 3s - loss: 0.0010 - acc: 1.0000    - ETA: 3s - loss: 9.4816e-04 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.0000    - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 2s - loss: 0.0011 - acc: 1.000 - ETA: 2s - loss: 0.0010 - acc: 1.000 - ETA: 2s - loss: 9.8808e-04 - acc: 1.000 - ETA: 2s - loss: 9.5852e-04 - acc: 1.000 - ETA: 2s - loss: 9.4

1808/1808 [==============================] - ETA: 4s - loss: 0.0023 - acc: 1.000 - ETA: 4s - loss: 0.0023 - acc: 1.000 - ETA: 4s - loss: 0.0023 - acc: 1.000 - ETA: 4s - loss: 0.0026 - acc: 1.000 - ETA: 4s - loss: 0.0033 - acc: 1.000 - ETA: 4s - loss: 0.0045 - acc: 1.000 - ETA: 4s - loss: 0.0044 - acc: 1.000 - ETA: 4s - loss: 0.0045 - acc: 1.000 - ETA: 4s - loss: 0.0056 - acc: 1.000 - ETA: 3s - loss: 0.0053 - acc: 1.000 - ETA: 3s - loss: 0.0051 - acc: 1.000 - ETA: 3s - loss: 0.0139 - acc: 0.997 - ETA: 3s - loss: 0.0132 - acc: 0.997 - ETA: 3s - loss: 0.0126 - acc: 0.997 - ETA: 3s - loss: 0.0120 - acc: 0.997 - ETA: 3s - loss: 0.0113 - acc: 0.998 - ETA: 3s - loss: 0.0108 - acc: 0.998 - ETA: 3s - loss: 0.0102 - acc: 0.998 - ETA: 3s - loss: 0.0099 - acc: 0.998 - ETA: 3s - loss: 0.0094 - acc: 0.998 - ETA: 2s - loss: 0.0091 - acc: 0.998 - ETA: 2s - loss: 0.0089 - acc: 0.998 - ETA: 2s - loss: 0.0087 - acc: 0.998 - ETA: 2s - loss: 0.0123 - acc: 0.997 - ETA: 2s - loss: 0.0119 - acc: 0.997 - ETA: 

...

1808/1808 [==============================] - ETA: 5s - loss: 0.0254 - acc: 0.968 - ETA: 4s - loss: 0.0127 - acc: 0.984 - ETA: 4s - loss: 0.0085 - acc: 0.989 - ETA: 4s - loss: 0.0064 - acc: 0.992 - ETA: 4s - loss: 0.0051 - acc: 0.993 - ETA: 4s - loss: 0.0042 - acc: 0.994 - ETA: 4s - loss: 0.0063 - acc: 0.995 - ETA: 4s - loss: 0.0055 - acc: 0.996 - ETA: 3s - loss: 0.0049 - acc: 0.996 - ETA: 3s - loss: 0.0044 - acc: 0.996 - ETA: 3s - loss: 0.0040 - acc: 0.997 - ETA: 3s - loss: 0.0037 - acc: 0.997 - ETA: 3s - loss: 0.0034 - acc: 0.997 - ETA: 3s - loss: 0.0032 - acc: 0.997 - ETA: 3s - loss: 0.0029 - acc: 0.997 - ETA: 3s - loss: 0.0028 - acc: 0.998 - ETA: 3s - loss: 0.0026 - acc: 0.998 - ETA: 3s - loss: 0.0025 - acc: 0.998 - ETA: 3s - loss: 0.0023 - acc: 0.998 - ETA: 3s - loss: 0.0022 - acc: 0.998 - ETA: 2s - loss: 0.0021 - acc: 0.998 - ETA: 2s - loss: 0.0020 - acc: 0.998 - ETA: 2s - loss: 0.0019 - acc: 0.998 - ETA: 2s - loss: 0.0018 - acc: 0.998 - ETA: 2s - loss: 0.0018 - acc: 0.998 - ETA: 

1808/1808 [==============================] - ETA: 4s - loss: 6.4634e-07 - acc: 1.000 - ETA: 4s - loss: 1.4752e-06 - acc: 1.000 - ETA: 4s - loss: 1.9365e-06 - acc: 1.000 - ETA: 4s - loss: 1.8180e-06 - acc: 1.000 - ETA: 4s - loss: 1.8288e-06 - acc: 1.000 - ETA: 4s - loss: 1.9490e-06 - acc: 1.000 - ETA: 4s - loss: 1.8651e-06 - acc: 1.000 - ETA: 4s - loss: 1.7753e-06 - acc: 1.000 - ETA: 3s - loss: 1.6776e-06 - acc: 1.000 - ETA: 3s - loss: 0.0015 - acc: 1.0000    - ETA: 3s - loss: 0.0031 - acc: 1.000 - ETA: 3s - loss: 0.0028 - acc: 1.000 - ETA: 3s - loss: 0.0026 - acc: 1.000 - ETA: 3s - loss: 0.0024 - acc: 1.000 - ETA: 3s - loss: 0.0022 - acc: 1.000 - ETA: 3s - loss: 0.0021 - acc: 1.000 - ETA: 3s - loss: 0.0020 - acc: 1.000 - ETA: 3s - loss: 0.0019 - acc: 1.000 - ETA: 3s - loss: 0.0018 - acc: 1.000 - ETA: 3s - loss: 0.0017 - acc: 1.000 - ETA: 3s - loss: 0.0025 - acc: 1.000 - ETA: 2s - loss: 0.0024 - acc: 1.000 - ETA: 2s - loss: 0.0023 - acc: 1.000 - ETA: 2s - loss: 0.0022 - acc: 1.000 - ETA

1808/1808 [==============================] - ETA: 5s - loss: 2.2017e-06 - acc: 1.000 - ETA: 5s - loss: 2.3246e-06 - acc: 1.000 - ETA: 5s - loss: 1.9161e-06 - acc: 1.000 - ETA: 4s - loss: 3.7879e-06 - acc: 1.000 - ETA: 4s - loss: 3.3920e-06 - acc: 1.000 - ETA: 4s - loss: 4.2992e-06 - acc: 1.000 - ETA: 4s - loss: 3.8168e-06 - acc: 1.000 - ETA: 4s - loss: 4.0093e-06 - acc: 1.000 - ETA: 4s - loss: 0.0027 - acc: 0.9965    - ETA: 4s - loss: 0.0025 - acc: 0.996 - ETA: 4s - loss: 0.0022 - acc: 0.997 - ETA: 3s - loss: 0.0036 - acc: 0.997 - ETA: 3s - loss: 0.0055 - acc: 0.995 - ETA: 3s - loss: 0.0051 - acc: 0.995 - ETA: 3s - loss: 0.0048 - acc: 0.995 - ETA: 3s - loss: 0.0045 - acc: 0.996 - ETA: 3s - loss: 0.0042 - acc: 0.996 - ETA: 3s - loss: 0.0040 - acc: 0.996 - ETA: 3s - loss: 0.0038 - acc: 0.996 - ETA: 3s - loss: 0.0036 - acc: 0.996 - ETA: 3s - loss: 0.0034 - acc: 0.997 - ETA: 3s - loss: 0.0033 - acc: 0.997 - ETA: 2s - loss: 0.0031 - acc: 0.997 - ETA: 2s - loss: 0.0030 - acc: 0.997 - ETA: 2s

1808/1808 [==============================] - ETA: 4s - loss: 1.0189e-06 - acc: 1.000 - ETA: 4s - loss: 0.0150 - acc: 0.9844    - ETA: 4s - loss: 0.0100 - acc: 0.989 - ETA: 4s - loss: 0.0121 - acc: 0.992 - ETA: 4s - loss: 0.0097 - acc: 0.993 - ETA: 4s - loss: 0.0081 - acc: 0.994 - ETA: 4s - loss: 0.0069 - acc: 0.995 - ETA: 4s - loss: 0.0061 - acc: 0.996 - ETA: 4s - loss: 0.0054 - acc: 0.996 - ETA: 4s - loss: 0.0049 - acc: 0.996 - ETA: 4s - loss: 0.0044 - acc: 0.997 - ETA: 3s - loss: 0.0069 - acc: 0.997 - ETA: 3s - loss: 0.0063 - acc: 0.997 - ETA: 3s - loss: 0.0059 - acc: 0.997 - ETA: 3s - loss: 0.0055 - acc: 0.997 - ETA: 3s - loss: 0.0052 - acc: 0.998 - ETA: 3s - loss: 0.0049 - acc: 0.998 - ETA: 3s - loss: 0.0046 - acc: 0.998 - ETA: 3s - loss: 0.0043 - acc: 0.998 - ETA: 3s - loss: 0.0041 - acc: 0.998 - ETA: 3s - loss: 0.0039 - acc: 0.998 - ETA: 3s - loss: 0.0037 - acc: 0.998 - ETA: 3s - loss: 0.0036 - acc: 0.998 - ETA: 2s - loss: 0.0034 - acc: 0.998 - ETA: 2s - loss: 0.0033 - acc: 0.998

1808/1808 [==============================] - ETA: 4s - loss: 0.0310 - acc: 0.968 - ETA: 4s - loss: 0.0155 - acc: 0.984 - ETA: 4s - loss: 0.0103 - acc: 0.989 - ETA: 4s - loss: 0.0078 - acc: 0.992 - ETA: 4s - loss: 0.0062 - acc: 0.993 - ETA: 4s - loss: 0.0052 - acc: 0.994 - ETA: 4s - loss: 0.0044 - acc: 0.995 - ETA: 3s - loss: 0.0039 - acc: 0.996 - ETA: 3s - loss: 0.0034 - acc: 0.996 - ETA: 3s - loss: 0.0031 - acc: 0.996 - ETA: 3s - loss: 0.0028 - acc: 0.997 - ETA: 3s - loss: 0.0026 - acc: 0.997 - ETA: 3s - loss: 0.0024 - acc: 0.997 - ETA: 3s - loss: 0.0022 - acc: 0.997 - ETA: 3s - loss: 0.0021 - acc: 0.997 - ETA: 3s - loss: 0.0031 - acc: 0.998 - ETA: 3s - loss: 0.0029 - acc: 0.998 - ETA: 3s - loss: 0.0028 - acc: 0.998 - ETA: 3s - loss: 0.0036 - acc: 0.998 - ETA: 3s - loss: 0.0034 - acc: 0.998 - ETA: 2s - loss: 0.0033 - acc: 0.998 - ETA: 2s - loss: 0.0031 - acc: 0.998 - ETA: 2s - loss: 0.0041 - acc: 0.997 - ETA: 2s - loss: 0.0039 - acc: 0.997 - ETA: 2s - loss: 0.0037 - acc: 0.997 - ETA: 

In [125]:
from keras.models import load_model
# load weights into new model
loaded_model= load_model("task3_cnn_fasttext.hdf5")

In [108]:
#model.evaluate(x_test,y_test)

453/453 [==============================] - 0s 317us/step


[1.327110879468602, 0.8487858720962574]

In [134]:
w=model.predict_classes(x_test)

In [135]:
from sklearn.metrics import classification_report,confusion_matrix

In [136]:
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [137]:
print(classification_report(Y_actual,w))

             precision    recall  f1-score   support

          0       0.10      0.10      0.10        40
          1       0.91      0.92      0.91       413

avg / total       0.84      0.84      0.84       453



In [138]:
print(confusion_matrix(Y_actual,w))

[[  4  36]
 [ 35 378]]


In [139]:
data_test=pd.read_csv('hasoc2019_en_test.tsv', sep = '\t', encoding="latin-1")

In [140]:
data_test.head()

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


In [150]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [151]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [152]:
w_test=model.predict_classes(X_t)

In [153]:
w_test

array([1, 0, 0, ..., 1, 1, 1], dtype=int64)

In [154]:
print(len(w_test)-np.count_nonzero(w_test))
np.count_nonzero(w_test)

110


1043

In [155]:
output = pd.DataFrame()
#out_result['text_id'] = 

In [156]:
output['text_id'] = data_test['text_id']

In [157]:
output.tail()

,text_id
1148,hasoc_en1_3958
1149,hasoc_en1_4648
1150,hasoc_en1_4832
1151,hasoc_en1_3721
1152,hasoc_en1_991


In [158]:
ans_dict = {0:'UNT',1:'TIN'}
w2 = np.vectorize(ans_dict.get)(w_test)
output['result'] = w2

In [159]:
output.to_csv('Kirti Kumari_English_task_3_run_3.tsv',header=True,sep='\t',index=False)